In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math
import random

import cv2
from sklearn.metrics import precision_score, recall_score, log_loss

import glob
import ntpath
from collections import Counter
import pickle
import copyreg
from tqdm import tqdm


In [8]:
%matplotlib notebook

In [91]:
def find_keypoints(path=r'/home/syakushin/DONE_SAME_SCALE/dress/*.jpg'):
    """Function that calculates all KAZE keypoints of images 
    corresponding to path template
    
    first part of the name before '_' (FIRSTPART_1_2_3.jpg) 
    is supposed to be similar for the photos of one thing

    Args:
        path: full path to folder with images. '*.jpg' is needed

    Returns:
        A list of (full_filename, keypoint_coordinate, keypoint_description)

    """
    kp_des = []
    kaze = cv2.KAZE_create()
    for filename_a in tqdm(glob.glob(path)):
        img1 = cv2.imread(filename_a, 0)
        kp1, des1 = kaze.detectAndCompute(img1,None)
        if (des1 is not None):
            kp_des.append((filename_a, kp1, des1))
        kp_des = sorted(kp_des, key = lambda x:x[0])
    return kp_des

In [9]:
def getComponents(normalised_homography):
  '''Calculates translationx, translationy, rotation, 
  scalex, scaley and  shear of homography matrix
  
    Args:
    normalised_homography: (3,3) numpy array

    Returns:
        translationx, translationy, rotation, 
        scalex, scaley and shear
  '''
  a = normalised_homography[0,0]
  b = normalised_homography[0,1]
  c = normalised_homography[0,2]
  d = normalised_homography[1,0]
  e = normalised_homography[1,1]
  f = normalised_homography[1,2]

  p = math.sqrt(a*a + b*b)
  r = (a*e - b*d)/(p)
  q = (a*d+b*e)/(a*e - b*d)

  translation_x, translation_y = (c,f)
  scale_x, scale_y = (p,r)
  shear = q
  theta = math.atan2(b,a)

  return (translation_x, translation_y, theta, scale_x, scale_y, shear)

In [89]:
def create_dataset_p(kp_des, MIN_MATCH_COUNT=4, compare_limit=256, negative_groups=1, ransacReprojThreshold=4):
    """Function that creates dataset from keypoints  of 'find_keypoints'
    
    Args:
        kp_des: keypoints from 'find_keypoints' output
        MIN_MATCH_COUNT: minimal number of keypoint matches to find a homography matrix
        compare_limit: max number of image comparison with images of particular item
        negative_groups: how many items are compared with another item. 
            Value 1 leads to more balanced dataset. 
            Value 0 leads to only positive examples.
        ransacReprojThreshold: findHomography parameter of RANSAC distance theshold

    Returns:
        X_data_bc: list of normalized features for each pair:
                number of matched pairs in homography
                translationx
                translationy
                rotation
                maximal  scale change
                scale change aspect ratio
                shear
                mean distance of matching keypoints description 
                max distance of matching keypoints description
                mean distance between homography and true keypoint 
                max distance between homography and true keypoint 
        y_data_b: list of true answers if a pair of images belongs to a single item 
        idx_data_b: indices of two images according to kp_des for described pairs
        idx_data_empty_b: indices of two images according to kp_des 
            for pairs with failed matching
        y_data_empty_b: list of true answers for pairs with failed matching 

    """
    kaze = cv2.KAZE_create()

    X_data_b = []
    y_data_b = []
    idx_data_b = []
    idx_data_empty_b = []
    y_data_empty_b = []                 

    kp_des_size = len(kp_des)

    for idx, (filename_a, kp1, des1) in tqdm(enumerate(kp_des)):
            next_touched = 0
            next_name = ''
            for idx_b in range(idx, np.minimum(kp_des_size, idx + compare_limit)):
                if next_touched <= negative_groups:
                    filename_b, kp2, des2 = kp_des[idx_b]
                    if not filename_a==filename_b:
                        if not ntpath.basename(filename_a).split("_")[0]==ntpath.basename(filename_b).split("_")[0]:
                            if next_touched < negative_groups:
                                next_touched += 1
                                next_name = ntpath.basename(filename_b).split("_")[0]
                            elif next_touched == negative_groups and not ntpath.basename(filename_b).split("_")[0]==next_name:
                                break
                        bf = cv2.BFMatcher(crossCheck=True)
                        idx_data_empty_b.append((idx, idx_b))
                        y_data_empty_b.append(int(int(ntpath.basename(filename_a).split("_")[0]) == int(ntpath.basename(filename_b).split("_")[0])))
                        if des2 is not None:
                            matches = bf.match(des1,des2)
                            matches = sorted(matches, key = lambda x:x.distance)
                            if len(matches) >= MIN_MATCH_COUNT:
                                src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
                                dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
                                features = []
                                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold)
                                
                                if M is not None:
                                    matchesMask = np.where(np.array(mask.ravel().tolist())==1)
                                    distances = np.abs([o.distance for o in matches])[matchesMask]
                                    match_max = np.max(distances)
                                    match_meansq = np.mean(distances)

                                    dst = np.array(cv2.perspectiveTransform(src_pts[matchesMask], M))

                                    dist = np.linalg.norm(dst_pts[matchesMask] - dst, axis=2)
                                    meansq_dist = np.mean(dist)
                                    max_dist = np.max(dist)
                                    translation_x, translation_y, rotation, scale_x, scale_y, shear = getComponents(M)
                                    features_i = [(np.sum(mask)-3)**(-1),
                                                  np.abs(translation_x)/2e6, 
                                                  np.abs(translation_y)/2e6, 
                                                  np.abs(rotation)/math.pi, 
                                                  np.min([np.abs(scale_x)**(-1), 
                                                         np.abs(scale_x),
                                                         np.abs(scale_y)**(-1), 
                                                         np.abs(scale_y)]), 
                                                  np.minimum(np.abs(scale_x/scale_y), np.abs(scale_y/scale_x)), 
                                                  np.abs(shear)/1e7,
                                                  match_max,
                                                  match_meansq,
                                                  0.1*meansq_dist/ransacReprojThreshold,
                                                  0.1*max_dist/ransacReprojThreshold]    
                                    features.append(features_i)
                                    X_data_b.append(features)
                                    y_data_b.append(int(int(ntpath.basename(filename_a).split("_")[0]) == int(ntpath.basename(filename_b).split("_")[0])))
                                    del y_data_empty_b[-1]
                                    idx_data_b.append((idx, idx_b))
                                    del idx_data_empty_b[-1]
    X_data_bc = np.array(X_data_b)

    return X_data_bc, y_data_b, idx_data_b, idx_data_empty_b, y_data_empty_b


In [12]:
def get_safe_balanced_split(target, trainSize=0.8, getTestIndexes=True, shuffle=False, seed=None):
    """Function that splits dataset to a balanced train and test
    
    Args:
        target: 'y' - answers
        trainSize:
        getTestIndexes: 
        shuffle: 
        seed: random seed

    Returns:
        trainIndexes, [testIndexes]

    """
    classes, counts = np.unique(target, return_counts=True)
    nPerClass = float(len(target))*float(trainSize)/float(len(classes))
    if nPerClass > np.min(counts):
        print("Insufficient data to produce a balanced training data split.")
        print("Classes found %s"%classes)
        print("Classes count %s"%counts)
        ts = float(trainSize*np.min(counts)*len(classes)) / float(len(target))
        print("trainSize is reset from %s to %s"%(trainSize, ts))
        trainSize = ts
        nPerClass = float(len(target))*float(trainSize)/float(len(classes))
    # get number of classes
    nPerClass = int(nPerClass)
    print("Data splitting on %i classes and returning %i per class"%(len(classes),nPerClass ))
    # get indexes
    trainIndexes = []
    for c in classes:
        if seed is not None:
            np.random.seed(seed)
        cIdxs = np.where(target==c)[0]
        cIdxs = np.random.choice(cIdxs, nPerClass, replace=False)
        trainIndexes.extend(cIdxs)
    # get test indexes
    testIndexes = None
    if getTestIndexes:
        testIndexes = list(set(range(len(target))) - set(trainIndexes))
    # shuffle
    if shuffle:
        trainIndexes = random.shuffle(trainIndexes)
        if testIndexes is not None:
            testIndexes = random.shuffle(testIndexes)
    # return indexes
    return trainIndexes, testIndexes

In [ ]:
kp_des = find_keypoints(r'/home/syakushin/DONE_SAME_SCALE/dress/*.jpg')

In [ ]:
kp_des_blouse = find_keypoints(r'/home/syakushin/DONE_SAME_SCALE/blouse/*.jpg')

In [90]:
X_data_dress_1_p, y_data_dress_1_p, idx_data_dress_1_p, idx_data_empty_dress_1_p, y_data_empty_dress_1_p = create_dataset_p(kp_des, MIN_MATCH_COUNT=4, compare_limit=256, negative_groups=2, ransacReprojThreshold=1)






0it [00:00, ?it/s]




1it [00:01,  1.94s/it]




2it [00:03,  1.87s/it]




3it [00:05,  1.84s/it]




4it [00:07,  1.85s/it]




5it [00:09,  1.93s/it]




6it [00:11,  1.94s/it]




7it [00:13,  1.89s/it]




8it [00:14,  1.86s/it]




9it [00:16,  1.80s/it]




10it [00:17,  1.75s/it]




11it [00:18,  1.70s/it]




12it [00:19,  1.65s/it]




13it [00:21,  1.64s/it]




14it [00:22,  1.61s/it]




15it [00:23,  1.58s/it]




16it [00:24,  1.52s/it]




17it [00:25,  1.49s/it]




18it [00:26,  1.47s/it]




19it [00:27,  1.45s/it]




20it [00:28,  1.42s/it]




21it [00:29,  1.39s/it]




22it [00:29,  1.36s/it]




23it [00:30,  1.32s/it]




24it [00:30,  1.29s/it]




25it [00:31,  1.25s/it]




26it [00:32,  1.24s/it]




27it [00:32,  1.21s/it]




28it [00:33,  1.19s/it]




29it [00:33,  1.17s/it]




30it [00:34,  1.15s/it]




31it [00:34,  1.13s/it]




32it [00:35,  1.10s/it]




33it [00:35,  1.08s/it]




34it [00:35,  1.05s/it]




35it [00:36,  1.05s/it]




3

380it [05:25,  1.17it/s]




381it [05:26,  1.17it/s]




382it [05:26,  1.17it/s]




383it [05:27,  1.17it/s]




384it [05:28,  1.17it/s]




385it [05:29,  1.17it/s]




386it [05:30,  1.17it/s]




387it [05:31,  1.17it/s]




388it [05:32,  1.17it/s]




389it [05:32,  1.17it/s]




390it [05:33,  1.17it/s]




391it [05:34,  1.17it/s]




392it [05:35,  1.17it/s]




393it [05:35,  1.17it/s]




394it [05:36,  1.17it/s]




395it [05:37,  1.17it/s]




396it [05:37,  1.17it/s]




397it [05:38,  1.17it/s]




398it [05:39,  1.17it/s]




399it [05:39,  1.17it/s]




400it [05:40,  1.17it/s]




401it [05:41,  1.18it/s]




402it [05:42,  1.18it/s]




403it [05:42,  1.18it/s]




404it [05:43,  1.18it/s]




405it [05:44,  1.18it/s]




406it [05:45,  1.18it/s]




407it [05:45,  1.18it/s]




408it [05:46,  1.18it/s]




409it [05:47,  1.18it/s]




410it [05:48,  1.18it/s]




411it [05:48,  1.18it/s]




412it [05:49,  1.18it/s]




413it [05:50,  1.18it/s]




414it [05:51, 

759it [11:40,  1.08it/s]




760it [11:41,  1.08it/s]




761it [11:42,  1.08it/s]




762it [11:43,  1.08it/s]




763it [11:43,  1.08it/s]




764it [11:46,  1.08it/s]




765it [11:48,  1.08it/s]




766it [11:50,  1.08it/s]




767it [11:52,  1.08it/s]




768it [11:54,  1.07it/s]




769it [11:56,  1.07it/s]




770it [11:58,  1.07it/s]




771it [12:00,  1.07it/s]




772it [12:02,  1.07it/s]




773it [12:04,  1.07it/s]




774it [12:06,  1.07it/s]




775it [12:08,  1.06it/s]




776it [12:10,  1.06it/s]




777it [12:11,  1.06it/s]




778it [12:14,  1.06it/s]




779it [12:17,  1.06it/s]




780it [12:19,  1.05it/s]




781it [12:23,  1.05it/s]




782it [12:25,  1.05it/s]




783it [12:27,  1.05it/s]




784it [12:29,  1.05it/s]




785it [12:31,  1.04it/s]




786it [12:33,  1.04it/s]




787it [12:34,  1.04it/s]




788it [12:36,  1.04it/s]




789it [12:37,  1.04it/s]




790it [12:38,  1.04it/s]




791it [12:39,  1.04it/s]




792it [12:40,  1.04it/s]




793it [12:41, 

1133it [18:19,  1.03it/s]




1134it [18:20,  1.03it/s]




1135it [18:20,  1.03it/s]




1136it [18:21,  1.03it/s]




1137it [18:22,  1.03it/s]




1138it [18:22,  1.03it/s]




1139it [18:23,  1.03it/s]




1140it [18:24,  1.03it/s]




1141it [18:25,  1.03it/s]




1142it [18:25,  1.03it/s]




1143it [18:26,  1.03it/s]




1144it [18:27,  1.03it/s]




1145it [18:27,  1.03it/s]




1146it [18:28,  1.03it/s]




1147it [18:28,  1.03it/s]




1148it [18:29,  1.03it/s]




1149it [18:30,  1.03it/s]




1150it [18:30,  1.04it/s]




1151it [18:31,  1.04it/s]




1152it [18:32,  1.04it/s]




1153it [18:32,  1.04it/s]




1154it [18:33,  1.04it/s]




1155it [18:34,  1.04it/s]




1156it [18:35,  1.04it/s]




1157it [18:36,  1.04it/s]




1158it [18:37,  1.04it/s]




1159it [18:37,  1.04it/s]




1160it [18:38,  1.04it/s]




1161it [18:39,  1.04it/s]




1162it [18:40,  1.04it/s]




1163it [18:41,  1.04it/s]




1164it [18:42,  1.04it/s]




1165it [18:43,  1.04it/s]




1166it [18

1500it [25:24,  1.02s/it]




1501it [25:25,  1.02s/it]




1502it [25:27,  1.02s/it]




1503it [25:29,  1.02s/it]




1504it [25:31,  1.02s/it]




1505it [25:32,  1.02s/it]




1506it [25:34,  1.02s/it]




1507it [25:35,  1.02s/it]




1508it [25:36,  1.02s/it]




1509it [25:38,  1.02s/it]




1510it [25:39,  1.02s/it]




1511it [25:40,  1.02s/it]




1512it [25:42,  1.02s/it]




1513it [25:42,  1.02s/it]




1514it [25:43,  1.02s/it]




1515it [25:44,  1.02s/it]




1516it [25:46,  1.02s/it]




1517it [25:47,  1.02s/it]




1518it [25:48,  1.02s/it]




1519it [25:50,  1.02s/it]




1520it [25:51,  1.02s/it]




1521it [25:52,  1.02s/it]




1522it [25:53,  1.02s/it]




1523it [25:54,  1.02s/it]




1524it [25:55,  1.02s/it]




1525it [25:56,  1.02s/it]




1526it [25:57,  1.02s/it]




1527it [25:58,  1.02s/it]




1528it [25:59,  1.02s/it]




1529it [26:00,  1.02s/it]




1530it [26:00,  1.02s/it]




1531it [26:01,  1.02s/it]




1532it [26:02,  1.02s/it]




1533it [26

...

4232it [1:14:15,  1.05s/it]




4233it [1:14:16,  1.05s/it]




4234it [1:14:16,  1.05s/it]




4235it [1:14:17,  1.05s/it]




4236it [1:14:18,  1.05s/it]




4238it [1:14:18,  1.05s/it]




4239it [1:14:19,  1.05s/it]




4240it [1:14:20,  1.05s/it]




4241it [1:14:21,  1.05s/it]




4242it [1:14:22,  1.05s/it]




4243it [1:14:23,  1.05s/it]




4244it [1:14:24,  1.05s/it]




4245it [1:14:24,  1.05s/it]




4246it [1:14:25,  1.05s/it]




4247it [1:14:26,  1.05s/it]




4248it [1:14:27,  1.05s/it]




4249it [1:14:27,  1.05s/it]




4250it [1:14:28,  1.05s/it]




4251it [1:14:29,  1.05s/it]




4252it [1:14:29,  1.05s/it]




4253it [1:14:30,  1.05s/it]




4254it [1:14:31,  1.05s/it]




4255it [1:14:32,  1.05s/it]




4256it [1:14:34,  1.05s/it]




4257it [1:14:35,  1.05s/it]




4258it [1:14:36,  1.05s/it]




4259it [1:14:37,  1.05s/it]




4260it [1:14:38,  1.05s/it]




4261it [1:14:39,  1.05s/it]




4262it [1:14:41,  1.05s/it]




4263it [1:14:43,  1.05s/it]




4264it [

4583it [1:20:30,  1.05s/it]




4584it [1:20:32,  1.05s/it]




4585it [1:20:33,  1.05s/it]




4586it [1:20:34,  1.05s/it]




4587it [1:20:35,  1.05s/it]




4588it [1:20:36,  1.05s/it]




4589it [1:20:37,  1.05s/it]




4590it [1:20:37,  1.05s/it]




4591it [1:20:38,  1.05s/it]




4592it [1:20:40,  1.05s/it]




4593it [1:20:41,  1.05s/it]




4594it [1:20:43,  1.05s/it]




4595it [1:20:44,  1.05s/it]




4596it [1:20:45,  1.05s/it]




4597it [1:20:47,  1.05s/it]




4598it [1:20:48,  1.05s/it]




4599it [1:20:49,  1.05s/it]




4600it [1:20:51,  1.05s/it]




4601it [1:20:52,  1.05s/it]




4602it [1:20:54,  1.05s/it]




4603it [1:20:55,  1.05s/it]




4604it [1:20:56,  1.05s/it]




4605it [1:20:58,  1.05s/it]




4606it [1:20:59,  1.06s/it]




4607it [1:21:00,  1.06s/it]




4608it [1:21:01,  1.06s/it]




4609it [1:21:02,  1.06s/it]




4610it [1:21:03,  1.05s/it]




4611it [1:21:04,  1.05s/it]




4612it [1:21:05,  1.05s/it]




4613it [1:21:06,  1.05s/it]




4614it [

4937it [1:26:55,  1.06s/it]




4938it [1:26:56,  1.06s/it]




4939it [1:26:56,  1.06s/it]




4940it [1:26:57,  1.06s/it]




4941it [1:26:58,  1.06s/it]




4942it [1:26:59,  1.06s/it]




4943it [1:27:00,  1.06s/it]




4944it [1:27:01,  1.06s/it]




4945it [1:27:02,  1.06s/it]




4946it [1:27:02,  1.06s/it]




4947it [1:27:03,  1.06s/it]




4948it [1:27:04,  1.06s/it]




4949it [1:27:05,  1.06s/it]




4950it [1:27:05,  1.06s/it]




4951it [1:27:06,  1.06s/it]




4952it [1:27:07,  1.06s/it]




4953it [1:27:08,  1.06s/it]




4954it [1:27:08,  1.06s/it]




4955it [1:27:09,  1.06s/it]




4956it [1:27:09,  1.06s/it]




4957it [1:27:10,  1.06s/it]




4958it [1:27:11,  1.06s/it]




4959it [1:27:12,  1.06s/it]




4960it [1:27:12,  1.06s/it]




4961it [1:27:13,  1.05s/it]




4962it [1:27:14,  1.05s/it]




4963it [1:27:14,  1.05s/it]




4964it [1:27:15,  1.05s/it]




4965it [1:27:16,  1.05s/it]




4966it [1:27:17,  1.05s/it]




4967it [1:27:18,  1.05s/it]




4968it [

5286it [1:32:45,  1.05s/it]




5287it [1:32:46,  1.05s/it]




5288it [1:32:47,  1.05s/it]




5289it [1:32:48,  1.05s/it]




5290it [1:32:48,  1.05s/it]




5291it [1:32:49,  1.05s/it]




5292it [1:32:50,  1.05s/it]




5293it [1:32:51,  1.05s/it]




5294it [1:32:51,  1.05s/it]




5295it [1:32:52,  1.05s/it]




5296it [1:32:53,  1.05s/it]




5297it [1:32:53,  1.05s/it]




5298it [1:32:55,  1.05s/it]




5299it [1:32:56,  1.05s/it]




5300it [1:32:57,  1.05s/it]




5301it [1:32:59,  1.05s/it]




5302it [1:33:00,  1.05s/it]




5303it [1:33:01,  1.05s/it]




5304it [1:33:02,  1.05s/it]




5305it [1:33:05,  1.05s/it]




5306it [1:33:06,  1.05s/it]




5307it [1:33:09,  1.05s/it]




5308it [1:33:11,  1.05s/it]




5309it [1:33:13,  1.05s/it]




5310it [1:33:15,  1.05s/it]




5311it [1:33:17,  1.05s/it]




5312it [1:33:18,  1.05s/it]




5313it [1:33:20,  1.05s/it]




5314it [1:33:22,  1.05s/it]




5315it [1:33:23,  1.05s/it]




5316it [1:33:25,  1.05s/it]




5317it [

5641it [1:37:34,  1.04s/it]




5642it [1:37:34,  1.04s/it]




5643it [1:37:35,  1.04s/it]




5644it [1:37:36,  1.04s/it]




5645it [1:37:36,  1.04s/it]




5646it [1:37:37,  1.04s/it]




5647it [1:37:38,  1.04s/it]




5648it [1:37:39,  1.04s/it]




5649it [1:37:41,  1.04s/it]




5650it [1:37:42,  1.04s/it]




5651it [1:37:43,  1.04s/it]




5652it [1:37:44,  1.04s/it]




5653it [1:37:45,  1.04s/it]




5654it [1:37:46,  1.04s/it]




5655it [1:37:48,  1.04s/it]




5656it [1:37:49,  1.04s/it]




5657it [1:37:50,  1.04s/it]




5658it [1:37:51,  1.04s/it]




5659it [1:37:52,  1.04s/it]




5660it [1:37:54,  1.04s/it]




5661it [1:37:54,  1.04s/it]




5662it [1:37:55,  1.04s/it]




5663it [1:37:56,  1.04s/it]




5664it [1:37:57,  1.04s/it]




5665it [1:37:58,  1.04s/it]




5666it [1:37:59,  1.04s/it]




5667it [1:37:59,  1.04s/it]




5668it [1:38:00,  1.04s/it]




5669it [1:38:00,  1.04s/it]




5670it [1:38:01,  1.04s/it]




5671it [1:38:02,  1.04s/it]




5672it [

In [92]:
np.save("X_data_dress_1_p", X_data_dress_1_p)
np.save("y_data_dress_1_p", y_data_dress_1_p)
np.save("idx_data_dress_1_p", idx_data_dress_1_p)
np.save("idx_data_empty_dress_1_p", idx_data_empty_dress_1_p)
np.save("y_data_empty_dress_1_p", y_data_empty_dress_1_p)

In [93]:
X_data_dress_4_p, y_data_dress_4_p, idx_data_dress_4_p, idx_data_empty_dress_4_p, y_data_empty_dress_4_p = create_dataset_p(kp_des, MIN_MATCH_COUNT=4, compare_limit=256, negative_groups=2, ransacReprojThreshold=4)






0it [00:00, ?it/s]




1it [00:02,  2.05s/it]




2it [00:04,  2.03s/it]




3it [00:05,  2.00s/it]




4it [00:08,  2.00s/it]




5it [00:10,  2.06s/it]




6it [00:12,  2.07s/it]




7it [00:14,  2.03s/it]




8it [00:15,  1.99s/it]




9it [00:17,  1.92s/it]




10it [00:18,  1.88s/it]




11it [00:20,  1.83s/it]




12it [00:21,  1.77s/it]




13it [00:22,  1.76s/it]




14it [00:24,  1.73s/it]




15it [00:25,  1.69s/it]




16it [00:25,  1.62s/it]




17it [00:27,  1.60s/it]




18it [00:28,  1.58s/it]




19it [00:29,  1.55s/it]




20it [00:30,  1.52s/it]




21it [00:31,  1.49s/it]




22it [00:32,  1.45s/it]




23it [00:32,  1.42s/it]




24it [00:33,  1.39s/it]




25it [00:33,  1.35s/it]




26it [00:34,  1.33s/it]




27it [00:35,  1.31s/it]




28it [00:35,  1.28s/it]




29it [00:36,  1.26s/it]




30it [00:37,  1.24s/it]




31it [00:37,  1.22s/it]




32it [00:38,  1.19s/it]




33it [00:38,  1.17s/it]




35it [00:39,  1.14s/it]




36it [00:40,  1.13s/it]




3

381it [05:25,  1.17it/s]




382it [05:25,  1.17it/s]




383it [05:26,  1.17it/s]




384it [05:27,  1.17it/s]




385it [05:28,  1.17it/s]




386it [05:29,  1.17it/s]




387it [05:30,  1.17it/s]




388it [05:30,  1.17it/s]




389it [05:31,  1.17it/s]




390it [05:32,  1.17it/s]




391it [05:33,  1.17it/s]




392it [05:33,  1.17it/s]




393it [05:34,  1.17it/s]




394it [05:35,  1.18it/s]




395it [05:35,  1.18it/s]




396it [05:36,  1.18it/s]




397it [05:37,  1.18it/s]




398it [05:38,  1.18it/s]




399it [05:38,  1.18it/s]




400it [05:39,  1.18it/s]




401it [05:39,  1.18it/s]




402it [05:40,  1.18it/s]




403it [05:41,  1.18it/s]




404it [05:42,  1.18it/s]




405it [05:43,  1.18it/s]




406it [05:44,  1.18it/s]




407it [05:44,  1.18it/s]




408it [05:45,  1.18it/s]




409it [05:45,  1.18it/s]




410it [05:46,  1.18it/s]




411it [05:47,  1.18it/s]




412it [05:48,  1.18it/s]




413it [05:48,  1.18it/s]




414it [05:49,  1.18it/s]




415it [05:50, 

761it [11:30,  1.10it/s]




762it [11:31,  1.10it/s]




763it [11:32,  1.10it/s]




764it [11:34,  1.10it/s]




765it [11:37,  1.10it/s]




766it [11:40,  1.09it/s]




767it [11:42,  1.09it/s]




768it [11:45,  1.09it/s]




769it [11:47,  1.09it/s]




770it [11:49,  1.09it/s]




771it [11:51,  1.08it/s]




772it [11:54,  1.08it/s]




773it [11:56,  1.08it/s]




774it [11:58,  1.08it/s]




775it [12:00,  1.08it/s]




776it [12:02,  1.07it/s]




777it [12:03,  1.07it/s]




778it [12:06,  1.07it/s]




779it [12:10,  1.07it/s]




780it [12:12,  1.06it/s]




781it [12:16,  1.06it/s]




782it [12:18,  1.06it/s]




783it [12:20,  1.06it/s]




784it [12:22,  1.06it/s]




785it [12:24,  1.05it/s]




786it [12:25,  1.05it/s]




787it [12:27,  1.05it/s]




788it [12:28,  1.05it/s]




789it [12:30,  1.05it/s]




790it [12:31,  1.05it/s]




791it [12:32,  1.05it/s]




792it [12:34,  1.05it/s]




793it [12:35,  1.05it/s]




794it [12:36,  1.05it/s]




795it [12:37, 

1135it [18:25,  1.03it/s]




1136it [18:26,  1.03it/s]




1137it [18:26,  1.03it/s]




1138it [18:27,  1.03it/s]




1139it [18:27,  1.03it/s]




1140it [18:28,  1.03it/s]




1141it [18:29,  1.03it/s]




1142it [18:30,  1.03it/s]




1143it [18:30,  1.03it/s]




1144it [18:31,  1.03it/s]




1145it [18:31,  1.03it/s]




1146it [18:32,  1.03it/s]




1147it [18:32,  1.03it/s]




1148it [18:33,  1.03it/s]




1149it [18:34,  1.03it/s]




1150it [18:35,  1.03it/s]




1151it [18:35,  1.03it/s]




1152it [18:36,  1.03it/s]




1153it [18:36,  1.03it/s]




1154it [18:37,  1.03it/s]




1155it [18:38,  1.03it/s]




1156it [18:39,  1.03it/s]




1157it [18:40,  1.03it/s]




1158it [18:41,  1.03it/s]




1159it [18:41,  1.03it/s]




1160it [18:42,  1.03it/s]




1161it [18:43,  1.03it/s]




1162it [18:44,  1.03it/s]




1163it [18:45,  1.03it/s]




1164it [18:46,  1.03it/s]




1165it [18:47,  1.03it/s]




1166it [18:48,  1.03it/s]




1167it [18:48,  1.03it/s]




1168it [18

1503it [24:45,  1.01it/s]




1504it [24:46,  1.01it/s]




1505it [24:48,  1.01it/s]




1506it [24:49,  1.01it/s]




1507it [24:50,  1.01it/s]




1508it [24:51,  1.01it/s]




1509it [24:53,  1.01it/s]




1510it [24:54,  1.01it/s]




1511it [24:55,  1.01it/s]




1512it [24:56,  1.01it/s]




1513it [24:56,  1.01it/s]




1514it [24:57,  1.01it/s]




1515it [24:58,  1.01it/s]




1516it [24:59,  1.01it/s]




1517it [25:00,  1.01it/s]




1518it [25:01,  1.01it/s]




1519it [25:03,  1.01it/s]




1520it [25:04,  1.01it/s]




1521it [25:05,  1.01it/s]




1522it [25:06,  1.01it/s]




1523it [25:06,  1.01it/s]




1524it [25:07,  1.01it/s]




1525it [25:08,  1.01it/s]




1526it [25:09,  1.01it/s]




1527it [25:10,  1.01it/s]




1528it [25:10,  1.01it/s]




1529it [25:11,  1.01it/s]




1530it [25:11,  1.01it/s]




1531it [25:12,  1.01it/s]




1532it [25:13,  1.01it/s]




1533it [25:14,  1.01it/s]




1534it [25:14,  1.01it/s]




1535it [25:15,  1.01it/s]




1536it [25

...

4066it [1:07:59,  1.00s/it]




4067it [1:08:00,  1.00s/it]




4068it [1:08:02,  1.00s/it]




4069it [1:08:03,  1.00s/it]




4070it [1:08:04,  1.00s/it]




4071it [1:08:05,  1.00s/it]




4072it [1:08:05,  1.00s/it]




4073it [1:08:06,  1.00s/it]




4074it [1:08:07,  1.00s/it]




4075it [1:08:07,  1.00s/it]




4076it [1:08:08,  1.00s/it]




4077it [1:08:09,  1.00s/it]




4078it [1:08:10,  1.00s/it]




4079it [1:08:11,  1.00s/it]




4080it [1:08:12,  1.00s/it]




4081it [1:08:13,  1.00s/it]




4082it [1:08:13,  1.00s/it]




4083it [1:08:14,  1.00s/it]




4085it [1:08:15,  1.00s/it]




4086it [1:08:16,  1.00s/it]




4087it [1:08:17,  1.00s/it]




4088it [1:08:18,  1.00s/it]




4089it [1:08:19,  1.00s/it]




4090it [1:08:20,  1.00s/it]




4091it [1:08:21,  1.00s/it]




4092it [1:08:21,  1.00s/it]




4093it [1:08:22,  1.00s/it]




4094it [1:08:23,  1.00s/it]




4095it [1:08:23,  1.00s/it]




4096it [1:08:24,  1.00s/it]




4097it [1:08:24,  1.00s/it]




4098it [

4425it [1:13:14,  1.01it/s]




4426it [1:13:15,  1.01it/s]




4427it [1:13:16,  1.01it/s]




4428it [1:13:18,  1.01it/s]




4429it [1:13:20,  1.01it/s]




4430it [1:13:21,  1.01it/s]




4431it [1:13:23,  1.01it/s]




4432it [1:13:24,  1.01it/s]




4433it [1:13:25,  1.01it/s]




4434it [1:13:26,  1.01it/s]




4435it [1:13:28,  1.01it/s]




4436it [1:13:29,  1.01it/s]




4437it [1:13:30,  1.01it/s]




4438it [1:13:30,  1.01it/s]




4439it [1:13:31,  1.01it/s]




4440it [1:13:32,  1.01it/s]




4441it [1:13:33,  1.01it/s]




4442it [1:13:34,  1.01it/s]




4443it [1:13:35,  1.01it/s]




4444it [1:13:36,  1.01it/s]




4445it [1:13:37,  1.01it/s]




4446it [1:13:38,  1.01it/s]




4447it [1:13:40,  1.01it/s]




4448it [1:13:41,  1.01it/s]




4449it [1:13:41,  1.01it/s]




4450it [1:13:43,  1.01it/s]




4451it [1:13:44,  1.01it/s]




4452it [1:13:45,  1.01it/s]




4453it [1:13:46,  1.01it/s]




4454it [1:13:47,  1.01it/s]




4455it [1:13:48,  1.01it/s]




4456it [

4776it [1:18:32,  1.01it/s]




4777it [1:18:33,  1.01it/s]




4778it [1:18:34,  1.01it/s]




4779it [1:18:35,  1.01it/s]




4780it [1:18:36,  1.01it/s]




4781it [1:18:37,  1.01it/s]




4782it [1:18:38,  1.01it/s]




4783it [1:18:39,  1.01it/s]




4784it [1:18:40,  1.01it/s]




4785it [1:18:40,  1.01it/s]




4786it [1:18:42,  1.01it/s]




4787it [1:18:43,  1.01it/s]




4788it [1:18:45,  1.01it/s]




4789it [1:18:46,  1.01it/s]




4790it [1:18:47,  1.01it/s]




4791it [1:18:49,  1.01it/s]




4792it [1:18:50,  1.01it/s]




4793it [1:18:52,  1.01it/s]




4794it [1:18:54,  1.01it/s]




4795it [1:18:56,  1.01it/s]




4796it [1:18:58,  1.01it/s]




4797it [1:19:01,  1.01it/s]




4798it [1:19:03,  1.01it/s]




4799it [1:19:05,  1.01it/s]




4800it [1:19:07,  1.01it/s]




4801it [1:19:09,  1.01it/s]




4802it [1:19:10,  1.01it/s]




4803it [1:19:12,  1.01it/s]




4804it [1:19:14,  1.01it/s]




4805it [1:19:15,  1.01it/s]




4806it [1:19:17,  1.01it/s]




4807it [

5128it [1:25:43,  1.00s/it]




5129it [1:25:44,  1.00s/it]




5130it [1:25:44,  1.00s/it]




5131it [1:25:45,  1.00s/it]




5132it [1:25:46,  1.00s/it]




5133it [1:25:47,  1.00s/it]




5134it [1:25:48,  1.00s/it]




5135it [1:25:49,  1.00s/it]




5136it [1:25:50,  1.00s/it]




5137it [1:25:51,  1.00s/it]




5138it [1:25:52,  1.00s/it]




5139it [1:25:52,  1.00s/it]




5140it [1:25:53,  1.00s/it]




5141it [1:25:54,  1.00s/it]




5142it [1:25:55,  1.00s/it]




5143it [1:25:56,  1.00s/it]




5144it [1:25:57,  1.00s/it]




5145it [1:25:58,  1.00s/it]




5146it [1:25:58,  1.00s/it]




5147it [1:25:59,  1.00s/it]




5148it [1:26:00,  1.00s/it]




5149it [1:26:01,  1.00s/it]




5150it [1:26:02,  1.00s/it]




5151it [1:26:03,  1.00s/it]




5152it [1:26:04,  1.00s/it]




5153it [1:26:04,  1.00s/it]




5154it [1:26:05,  1.00s/it]




5155it [1:26:06,  1.00s/it]




5156it [1:26:06,  1.00s/it]




5157it [1:26:07,  1.00s/it]




5158it [1:26:09,  1.00s/it]




5159it [

5479it [1:30:48,  1.01it/s]




5480it [1:30:49,  1.01it/s]




5481it [1:30:50,  1.01it/s]




5482it [1:30:50,  1.01it/s]




5483it [1:30:51,  1.01it/s]




5484it [1:30:51,  1.01it/s]




5485it [1:30:51,  1.01it/s]




5486it [1:30:52,  1.01it/s]




5487it [1:30:53,  1.01it/s]




5488it [1:30:54,  1.01it/s]




5489it [1:30:54,  1.01it/s]




5490it [1:30:55,  1.01it/s]




5492it [1:30:55,  1.01it/s]




5493it [1:30:56,  1.01it/s]




5494it [1:30:57,  1.01it/s]




5495it [1:30:58,  1.01it/s]




5496it [1:30:59,  1.01it/s]




5497it [1:30:59,  1.01it/s]




5498it [1:31:00,  1.01it/s]




5499it [1:31:01,  1.01it/s]




5500it [1:31:01,  1.01it/s]




5501it [1:31:02,  1.01it/s]




5502it [1:31:03,  1.01it/s]




5503it [1:31:04,  1.01it/s]




5504it [1:31:04,  1.01it/s]




5505it [1:31:05,  1.01it/s]




5506it [1:31:06,  1.01it/s]




5507it [1:31:06,  1.01it/s]




5508it [1:31:07,  1.01it/s]




5509it [1:31:08,  1.01it/s]




5510it [1:31:09,  1.01it/s]




5511it [

In [94]:
np.save("X_data_dress_4_p", X_data_dress_4_p)
np.save("y_data_dress_4_p", y_data_dress_4_p)
np.save("idx_data_dress_4_p", idx_data_dress_4_p)
np.save("idx_data_empty_dress_4_p", idx_data_empty_dress_4_p)
np.save("y_data_empty_dress_4_p", y_data_empty_dress_4_p)

In [96]:
X_data_dress_8_p, y_data_dress_8_p, idx_data_dress_8_p, idx_data_empty_dress_8_p, y_data_empty_dress_8_p = create_dataset_p(kp_des, MIN_MATCH_COUNT=4, compare_limit=256, negative_groups=2, ransacReprojThreshold=16)
np.save("X_data_dress_8_p", X_data_dress_8_p)
np.save("y_data_dress_8_p", y_data_dress_8_p)
np.save("idx_data_dress_8_p", idx_data_dress_8_p)
np.save("idx_data_empty_dress_8_p", idx_data_empty_dress_8_p)
np.save("y_data_empty_dress_8_p", y_data_empty_dress_8_p)






0it [00:00, ?it/s]




1it [00:01,  1.70s/it]




2it [00:03,  1.72s/it]




3it [00:05,  1.70s/it]




4it [00:06,  1.72s/it]




5it [00:08,  1.78s/it]




6it [00:10,  1.80s/it]




7it [00:12,  1.77s/it]




8it [00:13,  1.74s/it]




9it [00:15,  1.68s/it]




10it [00:16,  1.65s/it]




11it [00:17,  1.60s/it]




12it [00:18,  1.55s/it]




13it [00:20,  1.55s/it]




14it [00:21,  1.52s/it]




15it [00:22,  1.49s/it]




16it [00:22,  1.44s/it]




17it [00:23,  1.41s/it]




18it [00:24,  1.38s/it]




19it [00:25,  1.35s/it]




20it [00:26,  1.33s/it]




21it [00:27,  1.29s/it]




22it [00:27,  1.26s/it]




23it [00:28,  1.23s/it]




24it [00:28,  1.20s/it]




25it [00:29,  1.16s/it]




26it [00:29,  1.14s/it]




27it [00:30,  1.12s/it]




28it [00:30,  1.10s/it]




29it [00:31,  1.07s/it]




30it [00:31,  1.05s/it]




31it [00:31,  1.03s/it]




32it [00:32,  1.01s/it]




33it [00:32,  1.02it/s]




35it [00:33,  1.05it/s]




36it [00:34,  1.05it/s]




3

382it [04:45,  1.34it/s]




383it [04:46,  1.34it/s]




384it [04:47,  1.34it/s]




385it [04:47,  1.34it/s]




386it [04:48,  1.34it/s]




387it [04:49,  1.34it/s]




388it [04:51,  1.33it/s]




389it [04:53,  1.33it/s]




390it [04:53,  1.33it/s]




391it [04:54,  1.33it/s]




392it [04:54,  1.33it/s]




393it [04:55,  1.33it/s]




394it [04:55,  1.33it/s]




395it [04:56,  1.33it/s]




396it [04:56,  1.33it/s]




397it [04:57,  1.33it/s]




398it [04:58,  1.33it/s]




399it [04:58,  1.34it/s]




400it [04:59,  1.34it/s]




401it [04:59,  1.34it/s]




402it [05:00,  1.34it/s]




403it [05:00,  1.34it/s]




404it [05:01,  1.34it/s]




405it [05:02,  1.34it/s]




406it [05:02,  1.34it/s]




407it [05:03,  1.34it/s]




408it [05:03,  1.34it/s]




409it [05:04,  1.34it/s]




410it [05:04,  1.34it/s]




411it [05:05,  1.35it/s]




412it [05:05,  1.35it/s]




413it [05:06,  1.35it/s]




414it [05:07,  1.35it/s]




415it [05:07,  1.35it/s]




416it [05:07, 

762it [10:52,  1.17it/s]




763it [10:53,  1.17it/s]




764it [10:55,  1.17it/s]




765it [10:57,  1.16it/s]




766it [10:59,  1.16it/s]




767it [11:01,  1.16it/s]




768it [11:03,  1.16it/s]




769it [11:05,  1.15it/s]




770it [11:07,  1.15it/s]




771it [11:09,  1.15it/s]




772it [11:11,  1.15it/s]




773it [11:13,  1.15it/s]




774it [11:15,  1.15it/s]




775it [11:16,  1.15it/s]




776it [11:18,  1.14it/s]




777it [11:19,  1.14it/s]




778it [11:21,  1.14it/s]




779it [11:24,  1.14it/s]




780it [11:27,  1.14it/s]




781it [11:29,  1.13it/s]




782it [11:31,  1.13it/s]




783it [11:35,  1.13it/s]




784it [11:38,  1.12it/s]




785it [11:40,  1.12it/s]




786it [11:42,  1.12it/s]




787it [11:43,  1.12it/s]




788it [11:44,  1.12it/s]




789it [11:45,  1.12it/s]




790it [11:46,  1.12it/s]




791it [11:47,  1.12it/s]




792it [11:49,  1.12it/s]




793it [11:51,  1.11it/s]




794it [11:52,  1.11it/s]




795it [11:53,  1.11it/s]




796it [11:54, 

1133it [21:38,  1.15s/it]




1134it [21:38,  1.15s/it]




1135it [21:40,  1.15s/it]




1136it [21:41,  1.15s/it]




1137it [21:43,  1.15s/it]




1138it [21:44,  1.15s/it]




1139it [21:45,  1.15s/it]




1140it [21:47,  1.15s/it]




1141it [21:49,  1.15s/it]




1142it [21:50,  1.15s/it]




1143it [21:52,  1.15s/it]




1144it [21:53,  1.15s/it]




1145it [21:54,  1.15s/it]




1146it [21:56,  1.15s/it]




1147it [21:57,  1.15s/it]




1148it [21:58,  1.15s/it]




1149it [22:00,  1.15s/it]




1150it [22:02,  1.15s/it]




1151it [22:03,  1.15s/it]




1152it [22:05,  1.15s/it]




1153it [22:06,  1.15s/it]




1154it [22:07,  1.15s/it]




1155it [22:09,  1.15s/it]




1156it [22:12,  1.15s/it]




1157it [22:14,  1.15s/it]




1158it [22:16,  1.15s/it]




1159it [22:18,  1.15s/it]




1160it [22:20,  1.16s/it]




1161it [22:21,  1.16s/it]




1162it [22:24,  1.16s/it]




1163it [22:26,  1.16s/it]




1164it [22:29,  1.16s/it]




1165it [22:30,  1.16s/it]




1166it [22

1497it [33:15,  1.33s/it]




1498it [33:17,  1.33s/it]




1499it [33:18,  1.33s/it]




1500it [33:19,  1.33s/it]




1501it [33:21,  1.33s/it]




1502it [33:23,  1.33s/it]




1503it [33:24,  1.33s/it]




1504it [33:26,  1.33s/it]




1505it [33:27,  1.33s/it]




1506it [33:29,  1.33s/it]




1507it [33:30,  1.33s/it]




1508it [33:31,  1.33s/it]




1509it [33:32,  1.33s/it]




1510it [33:34,  1.33s/it]




1511it [33:35,  1.33s/it]




1512it [33:36,  1.33s/it]




1513it [33:36,  1.33s/it]




1514it [33:37,  1.33s/it]




1515it [33:38,  1.33s/it]




1516it [33:39,  1.33s/it]




1517it [33:41,  1.33s/it]




1518it [33:41,  1.33s/it]




1519it [33:43,  1.33s/it]




1520it [33:44,  1.33s/it]




1521it [33:45,  1.33s/it]




1522it [33:46,  1.33s/it]




1523it [33:47,  1.33s/it]




1524it [33:48,  1.33s/it]




1525it [33:49,  1.33s/it]




1526it [33:50,  1.33s/it]




1527it [33:51,  1.33s/it]




1528it [33:51,  1.33s/it]




1529it [33:52,  1.33s/it]




1530it [33

...

4049it [1:16:32,  1.13s/it]




4051it [1:16:33,  1.13s/it]




4052it [1:16:34,  1.13s/it]




4053it [1:16:36,  1.13s/it]




4054it [1:16:37,  1.13s/it]




4055it [1:16:39,  1.13s/it]




4056it [1:16:40,  1.13s/it]




4057it [1:16:41,  1.13s/it]




4058it [1:16:43,  1.13s/it]




4059it [1:16:45,  1.13s/it]




4060it [1:16:45,  1.13s/it]




4061it [1:16:47,  1.13s/it]




4062it [1:16:49,  1.13s/it]




4063it [1:16:51,  1.13s/it]




4064it [1:16:53,  1.14s/it]




4065it [1:16:54,  1.14s/it]




4066it [1:16:56,  1.14s/it]




4067it [1:16:57,  1.14s/it]




4068it [1:16:59,  1.14s/it]




4069it [1:17:00,  1.14s/it]




4070it [1:17:01,  1.14s/it]




4071it [1:17:02,  1.14s/it]




4072it [1:17:03,  1.14s/it]




4073it [1:17:04,  1.14s/it]




4074it [1:17:05,  1.14s/it]




4075it [1:17:06,  1.14s/it]




4076it [1:17:06,  1.14s/it]




4077it [1:17:07,  1.14s/it]




4078it [1:17:08,  1.14s/it]




4079it [1:17:09,  1.14s/it]




4080it [1:17:10,  1.14s/it]




4081it [

4409it [1:22:32,  1.12s/it]




4410it [1:22:34,  1.12s/it]




4411it [1:22:35,  1.12s/it]




4412it [1:22:37,  1.12s/it]




4413it [1:22:39,  1.12s/it]




4414it [1:22:41,  1.12s/it]




4415it [1:22:42,  1.12s/it]




4416it [1:22:44,  1.12s/it]




4417it [1:22:45,  1.12s/it]




4418it [1:22:47,  1.12s/it]




4419it [1:22:48,  1.12s/it]




4420it [1:22:50,  1.12s/it]




4421it [1:22:51,  1.12s/it]




4422it [1:22:52,  1.12s/it]




4423it [1:22:54,  1.12s/it]




4424it [1:22:55,  1.12s/it]




4425it [1:22:56,  1.12s/it]




4426it [1:22:57,  1.12s/it]




4427it [1:22:58,  1.12s/it]




4428it [1:23:00,  1.12s/it]




4429it [1:23:02,  1.12s/it]




4430it [1:23:03,  1.12s/it]




4431it [1:23:05,  1.13s/it]




4432it [1:23:06,  1.13s/it]




4433it [1:23:07,  1.13s/it]




4434it [1:23:08,  1.13s/it]




4435it [1:23:09,  1.13s/it]




4436it [1:23:10,  1.13s/it]




4437it [1:23:11,  1.13s/it]




4438it [1:23:12,  1.12s/it]




4439it [1:23:13,  1.12s/it]




4440it [

4760it [1:27:53,  1.11s/it]




4761it [1:27:55,  1.11s/it]




4762it [1:27:57,  1.11s/it]




4763it [1:27:58,  1.11s/it]




4764it [1:28:00,  1.11s/it]




4765it [1:28:01,  1.11s/it]




4766it [1:28:03,  1.11s/it]




4767it [1:28:05,  1.11s/it]




4768it [1:28:06,  1.11s/it]




4769it [1:28:07,  1.11s/it]




4770it [1:28:08,  1.11s/it]




4771it [1:28:10,  1.11s/it]




4772it [1:28:11,  1.11s/it]




4773it [1:28:13,  1.11s/it]




4774it [1:28:14,  1.11s/it]




4775it [1:28:15,  1.11s/it]




4776it [1:28:17,  1.11s/it]




4777it [1:28:18,  1.11s/it]




4778it [1:28:19,  1.11s/it]




4779it [1:28:20,  1.11s/it]




4780it [1:28:21,  1.11s/it]




4781it [1:28:22,  1.11s/it]




4782it [1:28:23,  1.11s/it]




4783it [1:28:24,  1.11s/it]




4784it [1:28:24,  1.11s/it]




4785it [1:28:25,  1.11s/it]




4786it [1:28:26,  1.11s/it]




4787it [1:28:28,  1.11s/it]




4788it [1:28:29,  1.11s/it]




4789it [1:28:31,  1.11s/it]




4790it [1:28:32,  1.11s/it]




4791it [

5113it [1:36:25,  1.13s/it]




5114it [1:36:29,  1.13s/it]




5115it [1:36:32,  1.13s/it]




5116it [1:36:35,  1.13s/it]




5117it [1:36:37,  1.13s/it]




5118it [1:36:38,  1.13s/it]




5119it [1:36:39,  1.13s/it]




5120it [1:36:41,  1.13s/it]




5121it [1:36:43,  1.13s/it]




5122it [1:36:45,  1.13s/it]




5123it [1:36:48,  1.13s/it]




5124it [1:36:50,  1.13s/it]




5125it [1:36:52,  1.13s/it]




5126it [1:36:54,  1.13s/it]




5127it [1:36:56,  1.13s/it]




5128it [1:36:57,  1.13s/it]




5129it [1:36:59,  1.13s/it]




5130it [1:37:00,  1.13s/it]




5131it [1:37:01,  1.13s/it]




5132it [1:37:04,  1.13s/it]




5133it [1:37:07,  1.14s/it]




5134it [1:37:09,  1.14s/it]




5135it [1:37:10,  1.14s/it]




5136it [1:37:12,  1.14s/it]




5137it [1:37:14,  1.14s/it]




5138it [1:37:16,  1.14s/it]




5139it [1:37:17,  1.14s/it]




5140it [1:37:19,  1.14s/it]




5141it [1:37:22,  1.14s/it]




5142it [1:37:23,  1.14s/it]




5143it [1:37:25,  1.14s/it]




5144it [

5465it [1:42:35,  1.13s/it]




5466it [1:42:36,  1.13s/it]




5467it [1:42:36,  1.13s/it]




5468it [1:42:37,  1.13s/it]




5470it [1:42:37,  1.13s/it]




5471it [1:42:38,  1.13s/it]




5472it [1:42:39,  1.13s/it]




5473it [1:42:40,  1.13s/it]




5474it [1:42:41,  1.13s/it]




5475it [1:42:41,  1.13s/it]




5476it [1:42:42,  1.13s/it]




5477it [1:42:43,  1.13s/it]




5478it [1:42:43,  1.13s/it]




5479it [1:42:44,  1.13s/it]




5480it [1:42:44,  1.12s/it]




5481it [1:42:45,  1.12s/it]




5482it [1:42:45,  1.12s/it]




5483it [1:42:46,  1.12s/it]




5484it [1:42:46,  1.12s/it]




5485it [1:42:46,  1.12s/it]




5486it [1:42:47,  1.12s/it]




5487it [1:42:48,  1.12s/it]




5488it [1:42:48,  1.12s/it]




5489it [1:42:49,  1.12s/it]




5490it [1:42:50,  1.12s/it]




5492it [1:42:50,  1.12s/it]




5493it [1:42:51,  1.12s/it]




5494it [1:42:52,  1.12s/it]




5495it [1:42:53,  1.12s/it]




5496it [1:42:53,  1.12s/it]




5497it [1:42:54,  1.12s/it]




5498it [

In [95]:
X_data_dress_16_p, y_data_dress_16_p, idx_data_dress_16_p, idx_data_empty_dress_16_p, y_data_empty_dress_16_p = create_dataset_p(kp_des, MIN_MATCH_COUNT=4, compare_limit=256, negative_groups=2, ransacReprojThreshold=16)
np.save("X_data_dress_16_p", X_data_dress_16_p)
np.save("y_data_dress_16_p", y_data_dress_16_p)
np.save("idx_data_dress_16_p", idx_data_dress_16_p)
np.save("idx_data_empty_dress_16_p", idx_data_empty_dress_16_p)
np.save("y_data_empty_dress_16_p", y_data_empty_dress_16_p)






0it [00:00, ?it/s]




1it [00:01,  1.68s/it]




2it [00:03,  1.72s/it]




3it [00:04,  1.66s/it]




4it [00:06,  1.68s/it]




5it [00:08,  1.73s/it]




6it [00:10,  1.75s/it]




7it [00:12,  1.73s/it]




8it [00:13,  1.69s/it]




9it [00:14,  1.63s/it]




10it [00:15,  1.60s/it]




11it [00:17,  1.56s/it]




12it [00:18,  1.51s/it]




13it [00:19,  1.50s/it]




14it [00:20,  1.48s/it]




15it [00:21,  1.45s/it]




16it [00:22,  1.39s/it]




17it [00:23,  1.36s/it]




18it [00:24,  1.33s/it]




19it [00:24,  1.31s/it]




20it [00:25,  1.29s/it]




21it [00:26,  1.25s/it]




22it [00:26,  1.22s/it]




23it [00:27,  1.19s/it]




24it [00:27,  1.16s/it]




25it [00:28,  1.13s/it]




26it [00:28,  1.11s/it]




27it [00:29,  1.09s/it]




28it [00:29,  1.07s/it]




29it [00:30,  1.04s/it]




30it [00:30,  1.02s/it]




31it [00:31,  1.00s/it]




32it [00:31,  1.02it/s]




33it [00:31,  1.05it/s]




35it [00:32,  1.08it/s]




36it [00:33,  1.08it/s]




3

384it [04:21,  1.47it/s]




385it [04:21,  1.47it/s]




386it [04:22,  1.47it/s]




387it [04:23,  1.47it/s]




388it [04:24,  1.47it/s]




389it [04:24,  1.47it/s]




390it [04:25,  1.47it/s]




391it [04:26,  1.47it/s]




392it [04:26,  1.47it/s]




393it [04:27,  1.47it/s]




394it [04:27,  1.47it/s]




395it [04:28,  1.47it/s]




396it [04:28,  1.47it/s]




397it [04:29,  1.47it/s]




398it [04:30,  1.47it/s]




399it [04:30,  1.47it/s]




400it [04:30,  1.48it/s]




401it [04:31,  1.48it/s]




402it [04:32,  1.48it/s]




403it [04:32,  1.48it/s]




404it [04:33,  1.48it/s]




405it [04:34,  1.48it/s]




406it [04:34,  1.48it/s]




407it [04:35,  1.48it/s]




408it [04:35,  1.48it/s]




409it [04:36,  1.48it/s]




410it [04:36,  1.48it/s]




411it [04:37,  1.48it/s]




412it [04:38,  1.48it/s]




413it [04:38,  1.48it/s]




414it [04:39,  1.48it/s]




415it [04:39,  1.48it/s]




416it [04:39,  1.49it/s]




417it [04:40,  1.49it/s]




418it [04:40, 

764it [10:12,  1.25it/s]




765it [10:14,  1.24it/s]




766it [10:16,  1.24it/s]




767it [10:18,  1.24it/s]




768it [10:20,  1.24it/s]




769it [10:22,  1.23it/s]




770it [10:25,  1.23it/s]




771it [10:27,  1.23it/s]




772it [10:29,  1.23it/s]




773it [10:31,  1.22it/s]




774it [10:33,  1.22it/s]




775it [10:35,  1.22it/s]




776it [10:37,  1.22it/s]




777it [10:39,  1.22it/s]




778it [10:41,  1.21it/s]




779it [10:45,  1.21it/s]




780it [10:47,  1.20it/s]




781it [10:51,  1.20it/s]




782it [10:53,  1.20it/s]




783it [10:56,  1.19it/s]




784it [10:58,  1.19it/s]




785it [11:00,  1.19it/s]




786it [11:02,  1.19it/s]




787it [11:03,  1.19it/s]




788it [11:05,  1.18it/s]




789it [11:06,  1.18it/s]




790it [11:07,  1.18it/s]




791it [11:09,  1.18it/s]




792it [11:10,  1.18it/s]




793it [11:11,  1.18it/s]




794it [11:11,  1.18it/s]




795it [11:13,  1.18it/s]




796it [11:14,  1.18it/s]




797it [11:15,  1.18it/s]




798it [11:16, 

1139it [16:40,  1.14it/s]




1140it [16:41,  1.14it/s]




1141it [16:41,  1.14it/s]




1142it [16:42,  1.14it/s]




1143it [16:42,  1.14it/s]




1144it [16:43,  1.14it/s]




1145it [16:44,  1.14it/s]




1146it [16:44,  1.14it/s]




1147it [16:44,  1.14it/s]




1148it [16:45,  1.14it/s]




1149it [16:46,  1.14it/s]




1150it [16:46,  1.14it/s]




1151it [16:47,  1.14it/s]




1152it [16:48,  1.14it/s]




1153it [16:48,  1.14it/s]




1154it [16:48,  1.14it/s]




1155it [16:49,  1.14it/s]




1156it [16:50,  1.14it/s]




1157it [16:51,  1.14it/s]




1158it [16:52,  1.14it/s]




1159it [16:53,  1.14it/s]




1160it [16:54,  1.14it/s]




1161it [16:54,  1.14it/s]




1162it [16:55,  1.14it/s]




1163it [16:56,  1.14it/s]




1164it [16:57,  1.14it/s]




1165it [16:58,  1.14it/s]




1166it [16:59,  1.14it/s]




1167it [17:00,  1.14it/s]




1168it [17:00,  1.14it/s]




1169it [17:01,  1.14it/s]




1170it [17:03,  1.14it/s]




1171it [17:05,  1.14it/s]




1172it [17

1508it [22:57,  1.09it/s]




1509it [22:58,  1.09it/s]




1510it [22:59,  1.09it/s]




1511it [23:00,  1.09it/s]




1512it [23:01,  1.09it/s]




1513it [23:01,  1.09it/s]




1514it [23:02,  1.10it/s]




1515it [23:03,  1.10it/s]




1516it [23:04,  1.10it/s]




1517it [23:05,  1.09it/s]




1518it [23:06,  1.10it/s]




1519it [23:07,  1.10it/s]




1520it [23:08,  1.10it/s]




1521it [23:08,  1.10it/s]




1522it [23:09,  1.10it/s]




1523it [23:10,  1.10it/s]




1524it [23:11,  1.10it/s]




1525it [23:12,  1.10it/s]




1526it [23:12,  1.10it/s]




1527it [23:13,  1.10it/s]




1528it [23:14,  1.10it/s]




1529it [23:14,  1.10it/s]




1530it [23:15,  1.10it/s]




1531it [23:15,  1.10it/s]




1532it [23:16,  1.10it/s]




1533it [23:17,  1.10it/s]




1534it [23:17,  1.10it/s]




1535it [23:18,  1.10it/s]




1536it [23:19,  1.10it/s]




1537it [23:19,  1.10it/s]




1538it [23:20,  1.10it/s]




1539it [23:20,  1.10it/s]




1540it [23:20,  1.10it/s]




1541it [23

...

4284it [1:05:46,  1.09it/s]




4285it [1:05:47,  1.09it/s]




4286it [1:05:49,  1.09it/s]




4287it [1:05:51,  1.08it/s]




4288it [1:05:52,  1.08it/s]




4289it [1:05:54,  1.08it/s]




4290it [1:05:55,  1.08it/s]




4291it [1:05:58,  1.08it/s]




4292it [1:06:01,  1.08it/s]




4293it [1:06:03,  1.08it/s]




4294it [1:06:06,  1.08it/s]




4295it [1:06:08,  1.08it/s]




4296it [1:06:11,  1.08it/s]




4297it [1:06:13,  1.08it/s]




4298it [1:06:15,  1.08it/s]




4299it [1:06:18,  1.08it/s]




4300it [1:06:20,  1.08it/s]




4301it [1:06:22,  1.08it/s]




4302it [1:06:24,  1.08it/s]




4303it [1:06:26,  1.08it/s]




4304it [1:06:27,  1.08it/s]




4305it [1:06:29,  1.08it/s]




4306it [1:06:30,  1.08it/s]




4307it [1:06:32,  1.08it/s]




4308it [1:06:33,  1.08it/s]




4309it [1:06:34,  1.08it/s]




4310it [1:06:35,  1.08it/s]




4311it [1:06:36,  1.08it/s]




4313it [1:06:37,  1.08it/s]




4314it [1:06:39,  1.08it/s]




4315it [1:06:40,  1.08it/s]




4316it [

4637it [1:11:32,  1.08it/s]




4638it [1:11:32,  1.08it/s]




4639it [1:11:33,  1.08it/s]




4640it [1:11:34,  1.08it/s]




4641it [1:11:34,  1.08it/s]




4642it [1:11:35,  1.08it/s]




4643it [1:11:36,  1.08it/s]




4644it [1:11:36,  1.08it/s]




4645it [1:11:37,  1.08it/s]




4646it [1:11:37,  1.08it/s]




4647it [1:11:38,  1.08it/s]




4648it [1:11:40,  1.08it/s]




4649it [1:11:41,  1.08it/s]




4650it [1:11:42,  1.08it/s]




4651it [1:11:44,  1.08it/s]




4652it [1:11:45,  1.08it/s]




4653it [1:11:46,  1.08it/s]




4654it [1:11:47,  1.08it/s]




4655it [1:11:48,  1.08it/s]




4656it [1:11:49,  1.08it/s]




4657it [1:11:50,  1.08it/s]




4658it [1:11:51,  1.08it/s]




4659it [1:11:52,  1.08it/s]




4660it [1:11:53,  1.08it/s]




4661it [1:11:54,  1.08it/s]




4662it [1:11:55,  1.08it/s]




4663it [1:11:56,  1.08it/s]




4664it [1:11:57,  1.08it/s]




4665it [1:12:00,  1.08it/s]




4666it [1:12:02,  1.08it/s]




4667it [1:12:03,  1.08it/s]




4668it [

4989it [1:19:45,  1.04it/s]




4990it [1:19:46,  1.04it/s]




4991it [1:19:47,  1.04it/s]




4992it [1:19:47,  1.04it/s]




4993it [1:19:48,  1.04it/s]




4994it [1:19:49,  1.04it/s]




4995it [1:19:49,  1.04it/s]




4996it [1:19:50,  1.04it/s]




4997it [1:19:50,  1.04it/s]




4998it [1:19:50,  1.04it/s]




4999it [1:19:51,  1.04it/s]




5000it [1:19:52,  1.04it/s]




5001it [1:19:53,  1.04it/s]




5002it [1:19:54,  1.04it/s]




5003it [1:19:55,  1.04it/s]




5004it [1:19:56,  1.04it/s]




5005it [1:19:57,  1.04it/s]




5006it [1:19:59,  1.04it/s]




5007it [1:20:01,  1.04it/s]




5008it [1:20:03,  1.04it/s]




5009it [1:20:06,  1.04it/s]




5010it [1:20:08,  1.04it/s]




5011it [1:20:10,  1.04it/s]




5012it [1:20:12,  1.04it/s]




5013it [1:20:15,  1.04it/s]




5014it [1:20:16,  1.04it/s]




5015it [1:20:18,  1.04it/s]




5016it [1:20:20,  1.04it/s]




5017it [1:20:22,  1.04it/s]




5018it [1:20:24,  1.04it/s]




5019it [1:20:25,  1.04it/s]




5020it [

5340it [1:28:35,  1.00it/s]




5341it [1:28:38,  1.00it/s]




5342it [1:28:41,  1.00it/s]




5343it [1:28:44,  1.00it/s]




5344it [1:28:46,  1.00it/s]




5345it [1:28:49,  1.00it/s]




5346it [1:28:51,  1.00it/s]




5347it [1:28:54,  1.00it/s]




5348it [1:28:55,  1.00it/s]




5349it [1:28:57,  1.00it/s]




5350it [1:28:59,  1.00it/s]




5351it [1:29:01,  1.00it/s]




5352it [1:29:03,  1.00it/s]




5353it [1:29:05,  1.00it/s]




5354it [1:29:06,  1.00it/s]




5355it [1:29:08,  1.00it/s]




5356it [1:29:09,  1.00it/s]




5357it [1:29:10,  1.00it/s]




5358it [1:29:13,  1.00it/s]




5359it [1:29:16,  1.00it/s]




5360it [1:29:19,  1.00it/s]




5361it [1:29:22,  1.00s/it]




5362it [1:29:25,  1.00s/it]




5363it [1:29:27,  1.00s/it]




5364it [1:29:30,  1.00s/it]




5365it [1:29:32,  1.00s/it]




5366it [1:29:34,  1.00s/it]




5367it [1:29:37,  1.00s/it]




5368it [1:29:39,  1.00s/it]




5369it [1:29:41,  1.00s/it]




5370it [1:29:43,  1.00s/it]




5371it [

5700it [1:33:56,  1.01it/s]




5701it [1:33:56,  1.01it/s]




5703it [1:33:57,  1.01it/s]




5704it [1:33:57,  1.01it/s]




5705it [1:33:58,  1.01it/s]




5706it [1:33:59,  1.01it/s]




5707it [1:34:00,  1.01it/s]




5708it [1:34:01,  1.01it/s]




5709it [1:34:01,  1.01it/s]




5710it [1:34:02,  1.01it/s]




5711it [1:34:02,  1.01it/s]




5712it [1:34:03,  1.01it/s]




5713it [1:34:03,  1.01it/s]




5714it [1:34:04,  1.01it/s]




5715it [1:34:05,  1.01it/s]




5716it [1:34:05,  1.01it/s]




5717it [1:34:06,  1.01it/s]




5718it [1:34:06,  1.01it/s]




5719it [1:34:06,  1.01it/s]




5720it [1:34:07,  1.01it/s]




5721it [1:34:08,  1.01it/s]




5722it [1:34:08,  1.01it/s]




5723it [1:34:09,  1.01it/s]




5724it [1:34:10,  1.01it/s]




5725it [1:34:10,  1.01it/s]




5726it [1:34:11,  1.01it/s]




5728it [1:34:11,  1.01it/s]




5729it [1:34:12,  1.01it/s]




5730it [1:34:13,  1.01it/s]




5731it [1:34:14,  1.01it/s]




5732it [1:34:15,  1.01it/s]




5733it [

In [97]:
from itertools import zip_longest

In [98]:
def find_first_diff(list1, list2):
    for index, (x, y) in enumerate(zip_longest(list1, list2, 
                                               fillvalue=object())):
        if x != y:
            return index

In [99]:
X_data = np.hstack((X_data_dress_1_p.tolist(), 
                    X_data_dress_4_p.tolist()[:12860] + X_data_dress_4_p.tolist()[12861:50469] + X_data_dress_4_p.tolist()[50470:57096] + X_data_dress_4_p.tolist()[57097:], 
                    X_data_dress_16_p.tolist()[:2596] + 
                    X_data_dress_16_p.tolist()[2597:12861]+
                    X_data_dress_16_p.tolist()[12862:38453]+
                    X_data_dress_16_p.tolist()[38454:50471]+
                    X_data_dress_16_p.tolist()[50472:55280]+
                    X_data_dress_16_p.tolist()[55281:57099]+
                    X_data_dress_16_p.tolist()[57100:88026]+
                    X_data_dress_16_p.tolist()[88027:]))
X_data = X_data.transpose(0,2,1).reshape(-1, 3*11)

In [100]:
y_data = np.array(y_data_dress_1_p)

In [109]:
trainIndexes, testIndexes = get_safe_balanced_split(y_data)

Insufficient data to produce a balanced training data split.
Classes found [0 1]
Classes count [58807 31729]
trainSize is reset from 0.8 to 0.5607316426614828
Data splitting on 2 classes and returning 25383 per class


In [110]:
X_train = [X_data[x] for x in trainIndexes]
y_train = [y_data[x] for x in trainIndexes]
X_test = [X_data[x] for x in testIndexes]
y_test = [y_data[x] for x in testIndexes]

In [111]:
np.random.seed(0)
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve

In [112]:
lr = LogisticRegression(solver='lbfgs', C=6, verbose=1, class_weight='balanced', max_iter=1000, n_jobs=8)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("recall = {}, precision = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr)))

[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:   21.8s finished


recall = 0.5885597226599433, precision = 0.2678187293847698


In [113]:
classifier_lsvc = LinearSVC(random_state=0, C=0.1, class_weight='balanced', max_iter=1000, tol=1e-6)
classifier_lsvc.fit(X_train, y_train)
y_pred_lr = classifier_lsvc.predict(X_test)
print("recall = {}, precision = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr)))

recall = 0.5824141191301607, precision = 0.2702742230347349


In [114]:
model_rf = RandomForestClassifier(n_estimators = 2000, n_jobs = -1, random_state =50, max_features = 9, min_samples_leaf = 5)
model_rf.fit(X_train, y_train)
y_pred_lr = model_rf.predict(X_test)
y_proba=model_rf.predict_proba(X_test)[:,1]
print("recall = {}, precision = {}, logloss = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr), log_loss(y_test, y_proba)))


recall = 0.5568862275449101, precision = 0.3237746220797068, logloss = 0.5892866711624382


In [119]:
X_data = np.hstack((X_data_dress_1_p.tolist(), 
                    X_data_dress_4_p.tolist()[:12860] + X_data_dress_4_p.tolist()[12861:50469] + X_data_dress_4_p.tolist()[50470:57096] + X_data_dress_4_p.tolist()[57097:], 
                    X_data_dress_8_p.tolist()[:2596] + 
                    X_data_dress_8_p.tolist()[2597:12861]+
                    X_data_dress_8_p.tolist()[12862:38453]+
                    X_data_dress_8_p.tolist()[38454:50471]+
                    X_data_dress_8_p.tolist()[50472:55280]+
                    X_data_dress_8_p.tolist()[55281:57099]+
                    X_data_dress_8_p.tolist()[57100:88026]+
                    X_data_dress_8_p.tolist()[88027:],
                    X_data_dress_16_p.tolist()[:2596] + 
                    X_data_dress_16_p.tolist()[2597:12861]+
                    X_data_dress_16_p.tolist()[12862:38453]+
                    X_data_dress_16_p.tolist()[38454:50471]+
                    X_data_dress_16_p.tolist()[50472:55280]+
                    X_data_dress_16_p.tolist()[55281:57099]+
                    X_data_dress_16_p.tolist()[57100:88026]+
                    X_data_dress_16_p.tolist()[88027:]))
X_data = X_data.transpose(0,2,1).reshape(-1, 4*11)

In [120]:
trainIndexes, testIndexes = get_safe_balanced_split(y_data)

Insufficient data to produce a balanced training data split.
Classes found [0 1]
Classes count [58807 31729]
trainSize is reset from 0.8 to 0.5607316426614828
Data splitting on 2 classes and returning 25383 per class


In [121]:
X_train = [X_data[x] for x in trainIndexes]
y_train = [y_data[x] for x in trainIndexes]
X_test = [X_data[x] for x in testIndexes]
y_test = [y_data[x] for x in testIndexes]

In [126]:
lr = LogisticRegression(solver='lbfgs', C=10, verbose=1, class_weight='balanced', max_iter=1000, n_jobs=8)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("recall = {}, precision = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr)))

[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:   29.6s finished


recall = 0.5877718247715096, precision = 0.26696249642141423


In [138]:
classifier_lsvc = LinearSVC(random_state=0, C=0.05, class_weight='balanced', max_iter=1000, tol=1e-6)
classifier_lsvc.fit(X_train, y_train)
y_pred_lr = classifier_lsvc.predict(X_test)
print("recall = {}, precision = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr)))

recall = 0.5820989599747872, precision = 0.2707218761451081


In [135]:
model_rf = RandomForestClassifier(n_estimators = 2000, n_jobs = -1, random_state =50, max_features = 44, min_samples_leaf = 5)
model_rf.fit(X_train, y_train)
y_pred_lr = model_rf.predict(X_test)
y_proba=model_rf.predict_proba(X_test)[:,1]
print("recall = {}, precision = {}, logloss = {}".format(recall_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr), log_loss(y_test, y_proba)))


recall = 0.5624015127639458, precision = 0.31522699169757995, logloss = 0.592270003260718


In [139]:
lr_0 = LogisticRegression(solver='lbfgs', C=6, verbose=1, class_weight='balanced', n_jobs=8)
lr_0.fit(np.array(X_train)[:,0:11], y_train)
lr_1 = LogisticRegression(solver='lbfgs', C=6, verbose=1, class_weight='balanced', n_jobs=8)
lr_1.fit(np.array(X_train)[:,11:22], y_train)
lr_2 = LogisticRegression(solver='lbfgs', C=6, verbose=1, class_weight='balanced', n_jobs=8)
lr_2.fit(np.array(X_train)[:,22:33], y_train)
y_pred_lr0 = lr_0.predict(np.array(X_test)[:,0:11])
y_pred_lr1 = lr_1.predict(np.array(X_test)[:,11:22])
y_pred_lr2 = lr_2.predict(np.array(X_test)[:,22:33])

model_rf0 = RandomForestClassifier(n_estimators = 2000,random_state =50, max_features = 9, min_samples_leaf = 5, n_jobs=8)
model_rf0.fit(np.array(X_train)[:,0:11], y_train)
model_rf1 = RandomForestClassifier(n_estimators = 2000, random_state =50, max_features = 9, min_samples_leaf = 5, n_jobs=8)
model_rf1.fit(np.array(X_train)[:,11:22], y_train)
model_rf2 = RandomForestClassifier(n_estimators = 2000, random_state =50, max_features = 9, min_samples_leaf = 5, n_jobs=8)
model_rf2.fit(np.array(X_train)[:,22:33], y_train)
y_pred_rf0 = model_rf0.predict(np.array(X_test)[:,0:11])
y_pred_rf1 = model_rf1.predict(np.array(X_test)[:,11:22])
y_pred_rf2 = model_rf2.predict(np.array(X_test)[:,22:33])

classifier_lsvc0 = LinearSVC(random_state=0, C=0.1, class_weight='balanced', max_iter=1000, tol=1e-6)
classifier_lsvc0.fit(np.array(X_train)[:,0:11], y_train)
classifier_lsvc1 = LinearSVC(random_state=0, C=0.1, class_weight='balanced', max_iter=1000, tol=1e-6)
classifier_lsvc1.fit(np.array(X_train)[:,11:22], y_train)
classifier_lsvc2 = LinearSVC(random_state=0, C=0.1, class_weight='balanced', max_iter=1000, tol=1e-6)
classifier_lsvc2.fit(np.array(X_train)[:,22:33], y_train)



y_proba0=lr_0.predict_proba(np.array(X_test)[:,0:11])[:,1]
y_proba1=lr_1.predict_proba(np.array(X_test)[:,11:22])[:,1]
y_proba2=lr_2.predict_proba(np.array(X_test)[:,22:33])[:,1]

y_probarf0=model_rf0.predict_proba(np.array(X_test)[:,0:11])[:,1]
y_probarf1=model_rf1.predict_proba(np.array(X_test)[:,11:22])[:,1]
y_probarf2=model_rf2.predict_proba(np.array(X_test)[:,22:33])[:,1]



y_tr_proba0=lr_0.predict_proba(np.array(X_train)[:,0:11])[:,1]
y_tr_proba1=lr_1.predict_proba(np.array(X_train)[:,11:22])[:,1]
y_tr_proba2=lr_2.predict_proba(np.array(X_train)[:,22:33])[:,1]

y_tr_probarf0=model_rf0.predict_proba(np.array(X_train)[:,0:11])[:,1]
y_tr_probarf1=model_rf1.predict_proba(np.array(X_train)[:,11:22])[:,1]
y_tr_probarf2=model_rf2.predict_proba(np.array(X_train)[:,22:33])[:,1]

y_pred_svc0 = classifier_lsvc0.predict(np.array(X_test)[:,0:11])
y_pred_svc1 = classifier_lsvc1.predict(np.array(X_test)[:,11:22])
y_pred_svc2 = classifier_lsvc2.predict(np.array(X_test)[:,22:33])

y_tr_pred_svc0 = classifier_lsvc0.predict(np.array(X_train)[:,0:11])
y_tr_pred_svc1 = classifier_lsvc1.predict(np.array(X_train)[:,11:22])
y_tr_pred_svc2 = classifier_lsvc2.predict(np.array(X_train)[:,22:33])

model_rf_common.fit(np.vstack(
(
y_tr_proba0,
y_tr_proba1,
y_tr_proba2,
y_tr_probarf0,
y_tr_probarf1,
y_tr_probarf2,
y_tr_pred_svc0,
y_tr_pred_svc1,
y_tr_pred_svc2
)
).T, y_train)

y_rf_common = model_rf_common.predict(np.vstack(
(
y_proba0,
y_proba1,
y_proba2,
y_probarf0,
y_probarf1,
y_probarf2,
y_pred_svc0,
y_pred_svc1,
y_pred_svc2
)).T)

y_proba_rf_common = model_rf_common.predict_proba(np.vstack(
(y_proba0,
y_proba1,
y_proba2,
y_probarf0,
y_probarf1,
y_probarf2,
y_pred_svc0,
y_pred_svc1,
y_pred_svc2
)).T)


print("recall = {}, precision = {}, logloss = {}".format(recall_score(y_test, y_rf_common), precision_score(y_test, y_rf_common), log_loss(y_test, y_proba_rf_common)))

recall = 0.43696816892530727, precision = 0.3443864878291108, logloss = 2.749522206370469


In [117]:
model2 = keras.models.Sequential()
model2.add(L.InputLayer(input_shape=[7]))
model2.add(L.BatchNormalization())
model2.add(L.Dense(64, activation='relu'))
model2.add(L.BatchNormalization())
model2.add(L.Dropout(0.1))
model2.add(L.Dense(64, activation='relu'))
model2.add(L.BatchNormalization())
model2.add(L.Dropout(0.1))
model2.add(L.Dense(64, activation='relu'))
model2.add(L.BatchNormalization())
model2.add(L.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy')

In [118]:
model2.fit(np.array(X_train_s), (np.array(y_train_s)), epochs=10000, validation_data=[(np.array(X_test_s)), (np.array(y_test_s))])

Train on 7838 samples, validate on 1960 samples
Epoch 1/10000
7838/7838 [==============================] - 6s 803us/step - loss: 0.7310 - val_loss: 0.6373
Epoch 2/10000
7838/7838 [==============================] - 2s 303us/step - loss: 0.7008 - val_loss: 0.6257
Epoch 3/10000
7838/7838 [==============================] - 2s 302us/step - loss: 0.6945 - val_loss: 0.6117
Epoch 4/10000
7838/7838 [==============================] - 2s 298us/step - loss: 0.6920 - val_loss: 0.6269
Epoch 5/10000
7838/7838 [==============================] - 2s 299us/step - loss: 0.6922 - val_loss: 0.6646
Epoch 6/10000
7838/7838 [==============================] - 2s 299us/step - loss: 0.6884 - val_loss: 0.6281
Epoch 7/10000
7838/7838 [==============================] - 2s 309us/step - loss: 0.6868 - val_loss: 0.6501
Epoch 8/10000
7838/7838 [==============================] - 2s 301us/step - loss: 0.6876 - val_loss: 0.6330
Epoch 9/10000
7838/7838 [==============================] - 2s 306us/step - loss: 0.6876 - val_lo

KeyboardInterrupt: 